---
# Section 5.5: Reduction to Hessenberg and Tridiagonal Forms
---

Given $A \in \mathbb{C}^{n \times n}$, we want to find a unitary matrix $Q$ such that

$$
A = Q B Q^*
$$

where $B$ is "simple" using a _finite_ algorithm.

Getting $B$ _triangular_ with a finite algorithm is not possible since that would violate the unsolvability of quintic polynomials.

Instead we will reduce $A$ to an **upper Hessenberg** matrix $B$:

$$
B =
\begin{bmatrix}
* & * & * & * & * \\
* & * & * & * & * \\
  & * & * & * & * \\
  &   & * & * & * \\
  &   &   & * & * \\
\end{bmatrix}.
$$

If $A = A^*$ then $B = B^*$ is **tridiagonl**:

$$
B =
\begin{bmatrix}
* & * &   &   &   \\
* & * & * &   &   \\
  & * & * & * &   \\
  &   & * & * & * \\
  &   &   & * & * \\
\end{bmatrix}.
$$



---

## A first attempt

In [ ]:
using LinearAlgebra

In [ ]:
function house(x)
    u = copy(x)
    
    τ = norm(x)
    if τ == 0.0
        γ = 0.0
    else
        if x[1] < 0
            τ = -τ    # τ = sign(x[1])*norm(x)
        end
        γ = τ + x[1]  # γ temporarily stores τ + x[1]
        u[1] = 1.0    # u normalized to u[1] = 1
        u[2:end] /= γ # divide u[2:end] by τ + x[1]
        γ /= τ        # γ = (τ + x[1])/τ
    end
    
    return u, γ, τ
end

housetimesleft(B::Matrix, u, γ) = B - (γ*u)*(u'*B)
housetimesright(B::Matrix, u, γ) = B - (B*u)*(γ*u')

In [ ]:
n = 5
A = rand(n,n)
B = copy(A)

In [ ]:
u, γ, τ = house(B[:,1])

In [ ]:
B = housetimesleft(B, u, γ)

In [ ]:
B = housetimesright(B, u, γ)

In [ ]:
[eigvals(A) eigvals(B)]

Although $A$ and $B$ have the same eigenvalues, the matrix $B$ is not "simple". We need a different approach.

---

## Reduction to Hessenberg Form

First we partition $A$ as

$$
A =
\begin{bmatrix}
a_{11} & c^T \\
b & \hat{A} \\
\end{bmatrix}.
$$

Let $\hat{Q}$ be the Householder reflector that gives us

$$
\hat{Q} b = 
\begin{bmatrix}
-\tau \\
0 \\
\end{bmatrix}.
$$

Now we let

$$
Q = 
\begin{bmatrix}
1 & 0 \\
0 & \hat{Q} \\
\end{bmatrix}.
$$

Note that $Q$ is unitary since $\hat{Q}$ is unitary.

Then

$$
QAQ = 
\begin{bmatrix}
1 & 0 \\
0 & \hat{Q} \\
\end{bmatrix}
\begin{bmatrix}
a_{11} & c^T \\
b & \hat{A} \\
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & \hat{Q} \\
\end{bmatrix} =
\begin{bmatrix}
a_{11} & c^T \hat{Q} \\
\hat{Q} b & \hat{Q}\hat{A}\hat{Q} \\
\end{bmatrix} = 
\left[
\begin{array}{c|c}
a_{11} & c^T \hat{Q} \\ \hline
\begin{matrix} -\tau \\ 0 \end{matrix} & \hat{Q}\hat{A}\hat{Q} \\
\end{array}
\right].
$$

Now the zeros in the first column have been preserved.

Repeat on columns $2, 3, \ldots, n-2$.

---

## `myhess` and `hessenberg`

In [ ]:
housetimes(x::Vector, u, γ) = x - (γ*dot(u, x))*u

function myhess(A)
    n = size(A,1)
    B = copy(A)
    for i=1:n-2
        u, γ, τ = house(B[i+1:end,i])
        B[i+1,i] = -τ
        B[i+2:end,i] .= 0
        B[i+1:end,i+1:end] = housetimesleft(B[i+1:end,i+1:end], u, γ)
        B[:,i+1:end] = housetimesright(B[:,i+1:end], u, γ)
    end
    return B
end
        

In [ ]:
n = 5
A = rand(n,n)

In [ ]:
B = myhess(A)

In [ ]:
F = hessenberg(A)

F.H

In [ ]:
B ≈ F.H

In [ ]:
eigvals(A) ≈ eigvals(B)

In [ ]:
n = 5
A = rand(n,n)
A = A + A'

In [ ]:
B = myhess(A)

In [ ]:
F = hessenberg(A)

F.H

In [ ]:
B ≈ Matrix(F.H)

In [ ]:
eigvals(A) ≈ eigvals(B)

---

## Flop count

Let $A \in \mathbb{R}^{n \times n}$. Then the total flops required to reduce $A$ to upper-Hessenberg form is

$$
\frac{10}{3} n^3 + O(n^2).
$$

If $A$ is symmetric, then the total flop count can be reduced to 

$$
\frac{4}{3} n^3 + O(n^2).
$$

---